In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Q3") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

22/12/18 14:17:41 WARN Utils: Your hostname, mostafa-UX410UQK resolves to a loopback address: 127.0.1.1; using 192.168.0.100 instead (on interface wlp2s0)
22/12/18 14:17:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/18 14:17:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/18 14:17:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
rddFromFile = sc.textFile("./records.txt") # Now you have a RDD with wikipedia posts
rddFromFile.take(1)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ']

In [3]:
baskets_rdd = rddFromFile.map(lambda x:list(set().union(x.split())))
## [[code1, code2,...],[..],... ]
baskets_rdd.take(2)

[['ELE17451', 'FRO11987', 'SNA90258', 'ELE89019', 'GRO99222'],
 ['ELE17451',
  'SNA11465',
  'FRO90334',
  'SNA80192',
  'SNA30755',
  'FRO12685',
  'GRO12298',
  'ELE91550',
  'FRO84225',
  'ELE26917',
  'ELE52966',
  'GRO99222']]

In [4]:
baskets_count =  baskets_rdd.count()
baskets_count

31101

# A-priori

In [5]:

one_items_count_rdd = baskets_rdd.flatMap(lambda x :[(item , 1) for item in x]).reduceByKey(lambda x,y:x+y)
# output 
## (key = item_code ,  value = countk )

## k=1

In [92]:
SUPPORT_THRESH = 500

one_frequent_items_count_rdd = one_items_count_rdd.filter(lambda x:x[1]>=SUPPORT_THRESH)
# output 
## (key = item_code ,  value = countk )
one_frequent_items_count = one_frequent_items_count_rdd.map(lambda x:((x[0],),x[1])).collectAsMap()

one_frequent_items_rdd = one_frequent_items_count_rdd.map(lambda x:x[0])

one_frequent_items = one_frequent_items_rdd.collect()

print(len(one_frequent_items))


141


In [93]:
frequent_items_count = one_frequent_items_count


In [94]:
from itertools import permutations

def remove_and_sort_replicates(x):
    
    if(isinstance(x[0], tuple)):
        x1 = x[0]
        x2 = x[1]

    else:
        x1 = [x[0]]
        x2 = x[1]
        
    if (any(item == x2 for item in x1) == False):
        item_list = list(x1)
        item_list.append(x2)
        item_list = tuple(sorted(item_list))
        return item_list
    else:
        return x1



def generate_rules_and_confidences(tuple_information):
    items_tuple , joint_count = tuple_information[0] , tuple_information[1]

    k = len(items_tuple)
    rules_list =[]
    for item in list(permutations(items_tuple,k)):
        for j in range(1 ,k ,1):
            if item[0:j] in frequent_items_count:
                rule = ((item[0:j] , item[j:] ), joint_count/frequent_items_count[item[0:j]])
                rules_list.append(rule)



    return rules_list





    

## k=2

In [ ]:


one_frequent_items_rdd = sc.parallelize(one_frequent_items)


k=2

two_frequent_candidates_rdd = one_frequent_items_rdd.cartesian(one_frequent_items_rdd).map(remove_and_sort_replicates).filter(lambda x:len(x)==k).distinct()
# output 1
## [((frequent_code1, frequent_code1)),((frequent_code1, frequent_code2)) ,..., ((frequent_code2, frequent_code1)),...]
# output 2
## [(frequent_code1, frequent_code2 ),(frequent_code1, frequent_code3) ,...]

two_frequent_items_count_rdd = two_frequent_candidates_rdd.cartesian(baskets_rdd).filter(lambda x: all(frequent_item in x[1] for frequent_item in x[0]))\
    .map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=SUPPORT_THRESH)


two_frequent_items_count = two_frequent_items_count_rdd.collectAsMap()
frequent_items_count.update(two_frequent_items_count)

# output 1&2
## [((frequent_code1, frequent_code2),[camera_code1, camera_code2,...]),((frequent_code1, frequent_code3),[camera_code1, camera_code2,...]) ,...]
# output 2
## [((frequent_code1, frequent_code2),[camera_code1, camera_code2,...]),((frequent_code1, frequent_code3),[camera_code1, camera_code2,...]) ,...]
# output 3
## [((frequent_code1, frequent_code2),1),((frequent_code1, frequent_code3),1) ,...]
# output 4 &5
## [((frequent_code1, frequent_code2),countk),((frequent_code1, frequent_code3),countk) ,...]





In [ ]:

print(two_frequent_items_count_rdd.flatMap(generate_rules_and_confidences).collectAsMap())

# output 
## [(((frequent_code1,) (frequent_code2,)),confidence),(((frequent_code1,) (frequent_code3,)),confidence) ,...]


{(('GRO73461',), ('SNA80324',)): 0.15602443087173792, (('SNA80324',), ('GRO73461',)): 0.18462549277266754, (('DAI62779',), ('FRO40251',)): 0.16049197540122995, (('FRO40251',), ('DAI62779',)): 0.27570213862406595, (('FRO40251',), ('GRO85051',)): 0.3125483122906467, (('GRO85051',), ('FRO40251',)): 0.999176276771005, (('ELE17451',), ('ELE32164',)): 0.1318709677419355, (('ELE32164',), ('ELE17451',)): 0.1792353560154332, (('ELE17451',), ('SNA80324',)): 0.15406451612903227, (('SNA80324',), ('ELE17451',)): 0.19612352168199737, (('DAI75645',), ('GRO73461',)): 0.260233918128655, (('GRO73461',), ('DAI75645',)): 0.19766796224319821, (('DAI62779',), ('ELE17451',)): 0.23878806059697016, (('ELE17451',), ('DAI62779',)): 0.41083870967741937, (('DAI62779',), ('ELE26917',)): 0.09749512524373781, (('ELE26917',), ('DAI62779',)): 0.2835951134380454, (('ELE17451',), ('GRO73461',)): 0.14967741935483872, (('GRO73461',), ('ELE17451',)): 0.16102165463631316, (('DAI43223',), ('ELE32164',)): 0.5511627906976744, (

## k=3

In [81]:

two_frequent_items_rdd = two_frequent_items_count_rdd.map(lambda x:x[0])


k=3

three_frequent_candidates_rdd = two_frequent_items_rdd.cartesian(one_frequent_items_rdd).map(remove_and_sort_replicates).filter(lambda x:len(x)==k).distinct()

three_frequent_items_count_rdd = three_frequent_candidates_rdd.cartesian(baskets_rdd).filter(lambda x: all(frequent_item in x[1] for frequent_item in x[0]))\
    .map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=SUPPORT_THRESH)


three_frequent_items_count = three_frequent_items_count_rdd.collectAsMap()
frequent_items_count.update(three_frequent_items_count)




In [82]:
three_frequent_items_count

{('DAI75645', 'FRO40251', 'SNA80324'): 550}

In [84]:
frequent_items_count

{('SNA90258',): 550,
 ('GRO73461',): 3602,
 ('DAI22896',): 1219,
 ('SNA69641',): 599,
 ('ELE59935',): 1311,
 ('DAI22177',): 1627,
 ('ELE66810',): 1697,
 ('DAI49199',): 664,
 ('GRO36567',): 832,
 ('SNA55952',): 1094,
 ('DAI48891',): 825,
 ('ELE11111',): 811,
 ('FRO92261',): 915,
 ('SNA66979',): 703,
 ('FRO16142',): 931,
 ('FRO24098',): 521,
 ('SNA59903',): 891,
 ('DAI55911',): 974,
 ('FRO31317',): 2330,
 ('SNA72163',): 1090,
 ('ELE20196',): 545,
 ('DAI63921',): 1773,
 ('SNA18336',): 709,
 ('DAI91290',): 1138,
 ('ELE12792',): 832,
 ('ELE86561',): 735,
 ('GRO85051',): 1214,
 ('DAI73122',): 566,
 ('FRO73056',): 728,
 ('ELE37770',): 630,
 ('ELE32164',): 2851,
 ('DAI88807',): 1316,
 ('FRO66272',): 927,
 ('SNA80324',): 3044,
 ('SNA88283',): 865,
 ('DAI75645',): 2736,
 ('SNA12663',): 523,
 ('SNA96271',): 1295,
 ('ELE49801',): 663,
 ('GRO50832',): 608,
 ('GRO68067',): 651,
 ('FRO85978',): 1918,
 ('SNA24799',): 504,
 ('SNA38068',): 811,
 ('ELE74009',): 1816,
 ('GRO56726',): 1784,
 ('FRO89565',):

In [86]:

print(three_frequent_items_count_rdd.flatMap(generate_rules_and_confidences).collectAsMap())

# output 
## [(rule ,confidence),(rule2 ,confidence) ,...]


{(('DAI75645',), ('FRO40251', 'SNA80324')): 0.2010233918128655, (('DAI75645', 'FRO40251'), ('SNA80324',)): 0.43859649122807015, (('DAI75645',), ('SNA80324', 'FRO40251')): 0.2010233918128655, (('DAI75645', 'SNA80324'), ('FRO40251',)): 0.48672566371681414, (('FRO40251',), ('DAI75645', 'SNA80324')): 0.14171605256377223, (('FRO40251',), ('SNA80324', 'DAI75645')): 0.14171605256377223, (('FRO40251', 'SNA80324'), ('DAI75645',)): 0.3895184135977337, (('SNA80324',), ('DAI75645', 'FRO40251')): 0.18068331143232588, (('SNA80324',), ('FRO40251', 'DAI75645')): 0.18068331143232588}
